In [1]:
# imports 
import pandas as pd
import numpy as np 
from data_extraction import DataExtractionHelper

# load helper 
helper = DataExtractionHelper()
# remove max column for display output 
pd.set_option('display.max_columns', None)

In [2]:
# read in gwas raw data
gwas_raw = helper.read_in_data('../data/gwas_cancer_revised.csv', 'csv')
gwas_raw

,PUBMEDID,JOURNAL,LINK,STUDY,DISEASE/TRAIT,REPORTED GENE(S),MAPPED_GENE,UPSTREAM_GENE_ID,DOWNSTREAM_GENE_ID,SNP_GENE_IDS,STRONGEST SNP-RISK ALLELE,SNPS,SNP_ID_CURRENT,CONTEXT,CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION
0,26151821,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/26151821,Genome-wide association study of colorectal ca...,Colorectal cancer,"SLC25A26, LRIG1",LRIG1,NaN,NaN,ENSG00000144749,rs812481-G,rs812481,812481,intron_variant,N,colorectal cancer,http://www.ebi.ac.uk/efo/EFO_0005842,GCST003017
1,26151821,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/26151821,Genome-wide association study of colorectal ca...,Colorectal cancer,NOS1,NOS1,NaN,NaN,ENSG00000089250,rs73208120-G,rs73208120,73208120,intron_variant,N,colorectal cancer,http://www.ebi.ac.uk/efo/EFO_0005842,GCST003017
2,26151821,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/26151821,Genome-wide association study of colorectal ca...,Colorectal cancer,"ENTPD7, COX15, CUTC, ABCC2, SLC25A28",NKX2-3 - SLC25A28,ENSG00000119919,ENSG00000155287,NaN,rs11190164-G,rs11190164,11190164,intergenic_variant,N,colorectal cancer,http://www.ebi.ac.uk/efo/EFO_0005842,GCST003017
3,26151821,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/26151821,Genome-wide association study of colorectal ca...,Colorectal cancer,"CUX2, BRAP, ACAD10, SH2B3","ATXN2, SH2B3",NaN,NaN,"ENSG00000204842, ENSG00000111252",rs3184504-C,rs3184504,3184504,missense_variant,N,colorectal cancer,http://www.ebi.ac.uk/efo/EFO_0005842,GCST003017
4,26151821,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/26151821,Genome-wide association study of colorectal ca...,Colorectal cancer,NR,DCBLD1,NaN,NaN,ENSG00000164465,rs4946260-T,rs4946260,4946260,intron_variant,N,colorectal cancer,http://www.ebi.ac.uk/efo/EFO_0005842,GCST003017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8242,34930913,Cell Discov,www.ncbi.nlm.nih.gov/pubmed/34930913,Genome-wide association and functional interro...,ovarian cancer,NaN,ENPP7P1,NaN,NaN,ENSG00000249188,rs142897723-GA,rs142897723,57874171,intron_variant,N,"overall survival, ovarian carcinoma","http://www.ebi.ac.uk/efo/EFO_0000638, http://w...",GCST90102289
8243,35658861,BMC Med,www.ncbi.nlm.nih.gov/pubmed/35658861,"Associations of genetic risk, BMI trajectories...",lung cancer,NaN,SLC16A7,NaN,NaN,ENSG00000118596,rs79297227-C,rs79297227,79297227,intron_variant,N,"longitudinal BMI measurement, non-small cell l...","http://www.ebi.ac.uk/efo/EFO_0005937, http://w...",GCST90132908
8244,35658861,BMC Med,www.ncbi.nlm.nih.gov/pubmed/35658861,"Associations of genetic risk, BMI trajectories...",lung cancer,NaN,LINC01811,NaN,NaN,ENSG00000226320,rs2336652-A,rs2336652,2336652,intron_variant,N,"longitudinal BMI measurement, non-small cell l...","http://www.ebi.ac.uk/efo/EFO_0005937, http://w...",GCST90132908
8245,35658861,BMC Med,www.ncbi.nlm.nih.gov/pubmed/35658861,"Associations of genetic risk, BMI trajectories...",lung cancer,NaN,CACNA1A,NaN,NaN,ENSG00000141837,rs16018-G,rs16018,16018,intron_variant,N,"longitudinal BMI measurement, non-small cell l...","http://www.ebi.ac.uk/efo/EFO_0005937, http://w...",GCST90132908


In [3]:
# drop columns that have 'NO MAPPED GENES' in the MAPPED_GENE column 
gwas_df = helper.split_col_on_delim(df = gwas_raw, split_column = 'MAPPED_GENE', new_col_name = 'biomarker', delim = ',')
valid_mapped_gene_condition = ~gwas_df['biomarker'].str.contains('no mapped genes', case = False, na = False)
gwas_df = gwas_df[valid_mapped_gene_condition]
gwas_df.reset_index(drop = True, inplace = True)
gwas_df

,PUBMEDID,JOURNAL,LINK,STUDY,DISEASE/TRAIT,REPORTED GENE(S),UPSTREAM_GENE_ID,DOWNSTREAM_GENE_ID,SNP_GENE_IDS,STRONGEST SNP-RISK ALLELE,SNPS,SNP_ID_CURRENT,CONTEXT,CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,biomarker
0,26151821,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/26151821,Genome-wide association study of colorectal ca...,Colorectal cancer,"SLC25A26, LRIG1",NaN,NaN,ENSG00000144749,rs812481-G,rs812481,812481,intron_variant,N,colorectal cancer,http://www.ebi.ac.uk/efo/EFO_0005842,GCST003017,LRIG1
1,26151821,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/26151821,Genome-wide association study of colorectal ca...,Colorectal cancer,NOS1,NaN,NaN,ENSG00000089250,rs73208120-G,rs73208120,73208120,intron_variant,N,colorectal cancer,http://www.ebi.ac.uk/efo/EFO_0005842,GCST003017,NOS1
2,26151821,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/26151821,Genome-wide association study of colorectal ca...,Colorectal cancer,"ENTPD7, COX15, CUTC, ABCC2, SLC25A28",ENSG00000119919,ENSG00000155287,NaN,rs11190164-G,rs11190164,11190164,intergenic_variant,N,colorectal cancer,http://www.ebi.ac.uk/efo/EFO_0005842,GCST003017,NKX2-3 - SLC25A28
3,26151821,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/26151821,Genome-wide association study of colorectal ca...,Colorectal cancer,"CUX2, BRAP, ACAD10, SH2B3",NaN,NaN,"ENSG00000204842, ENSG00000111252",rs3184504-C,rs3184504,3184504,missense_variant,N,colorectal cancer,http://www.ebi.ac.uk/efo/EFO_0005842,GCST003017,ATXN2
4,26151821,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/26151821,Genome-wide association study of colorectal ca...,Colorectal cancer,"CUX2, BRAP, ACAD10, SH2B3",NaN,NaN,"ENSG00000204842, ENSG00000111252",rs3184504-C,rs3184504,3184504,missense_variant,N,colorectal cancer,http://www.ebi.ac.uk/efo/EFO_0005842,GCST003017,SH2B3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9743,34930913,Cell Discov,www.ncbi.nlm.nih.gov/pubmed/34930913,Genome-wide association and functional interro...,ovarian cancer,NaN,NaN,NaN,ENSG00000249188,rs142897723-GA,rs142897723,57874171,intron_variant,N,"overall survival, ovarian carcinoma","http://www.ebi.ac.uk/efo/EFO_0000638, http://w...",GCST90102289,ENPP7P1
9744,35658861,BMC Med,www.ncbi.nlm.nih.gov/pubmed/35658861,"Associations of genetic risk, BMI trajectories...",lung cancer,NaN,NaN,NaN,ENSG00000118596,rs79297227-C,rs79297227,79297227,intron_variant,N,"longitudinal BMI measurement, non-small cell l...","http://www.ebi.ac.uk/efo/EFO_0005937, http://w...",GCST90132908,SLC16A7
9745,35658861,BMC Med,www.ncbi.nlm.nih.gov/pubmed/35658861,"Associations of genetic risk, BMI trajectories...",lung cancer,NaN,NaN,NaN,ENSG00000226320,rs2336652-A,rs2336652,2336652,intron_variant,N,"longitudinal BMI measurement, non-small cell l...","http://www.ebi.ac.uk/efo/EFO_0005937, http://w...",GCST90132908,LINC01811
9746,35658861,BMC Med,www.ncbi.nlm.nih.gov/pubmed/35658861,"Associations of genetic risk, BMI trajectories...",lung cancer,NaN,NaN,NaN,ENSG00000141837,rs16018-G,rs16018,16018,intron_variant,N,"longitudinal BMI measurement, non-small cell l...","http://www.ebi.ac.uk/efo/EFO_0005937, http://w...",GCST90132908,CACNA1A


In [4]:
def split_x_rows(row):
    ''' Split x formatted rows and the corresponding rs id's. 
    '''
    biomarkers = row['biomarker'].split(' x ')
    snps = row['SNPS'].split(' x ')

    new_row = row.copy()
    row['biomarker'] = biomarkers[0]
    row['SNPS'] = snps[0]
    new_row['biomarker'] = biomarkers[1]
    new_row['SNPS'] = snps[1]
    return pd.DataFrame([row, new_row])

# extract rows with ' x ' formatted genes
x_mask = gwas_df['biomarker'].str.contains(' x ', case = False, na = False)
x_rows = gwas_df[x_mask]

# generate new formatted ' x ' rows 
new_x_rows = x_rows.apply(split_x_rows, axis = 1).tolist()
x_split_rows = pd.concat(new_x_rows, ignore_index = True)

# put the newly formatted x rows back into the dataset 
gwas_df = gwas_df[~x_mask]
gwas_df = pd.concat([gwas_df, x_split_rows])
gwas_df.reset_index(drop = True, inplace = True)
gwas_df.shape

(9767, 18)

In [5]:
# drop rows that have a ';' in the mapped gene data
gwas_df.drop(index = list(gwas_df[gwas_df['biomarker'].str.contains(';')].index), inplace = True)
gwas_df.reset_index(drop = True, inplace = True)
gwas_df[gwas_df['biomarker'].str.contains(';')]

,PUBMEDID,JOURNAL,LINK,STUDY,DISEASE/TRAIT,REPORTED GENE(S),UPSTREAM_GENE_ID,DOWNSTREAM_GENE_ID,SNP_GENE_IDS,STRONGEST SNP-RISK ALLELE,SNPS,SNP_ID_CURRENT,CONTEXT,CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,biomarker


In [6]:
# drop rows that have a range listed in the mapped gene data 
gwas_df.drop(index = list(gwas_df[gwas_df['biomarker'].str.contains(' - ')].index), inplace = True)
gwas_df.reset_index(drop = True, inplace = True)
gwas_df[gwas_df['biomarker'].str.contains(' - ')]

,PUBMEDID,JOURNAL,LINK,STUDY,DISEASE/TRAIT,REPORTED GENE(S),UPSTREAM_GENE_ID,DOWNSTREAM_GENE_ID,SNP_GENE_IDS,STRONGEST SNP-RISK ALLELE,SNPS,SNP_ID_CURRENT,CONTEXT,CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,biomarker


In [7]:
# split on the disease column when a list is provided
gwas_df = helper.split_col_on_delim(df = gwas_df, split_column = 'DISEASE/TRAIT', new_col_name = 'disease', delim = ',')
gwas_df.reset_index(drop = True, inplace = True)
gwas_df.shape

(7125, 18)

In [8]:
# drop rows that have x formatted SNPS values 
gwas_df.drop(index = list(gwas_df[gwas_df['SNPS'].str.contains(' x ')].index), inplace = True)
gwas_df.reset_index(drop = True, inplace = True)
gwas_df[gwas_df['SNPS'].str.contains(' x ')]

,PUBMEDID,JOURNAL,LINK,STUDY,REPORTED GENE(S),UPSTREAM_GENE_ID,DOWNSTREAM_GENE_ID,SNP_GENE_IDS,STRONGEST SNP-RISK ALLELE,SNPS,SNP_ID_CURRENT,CONTEXT,CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,biomarker,disease


In [9]:
# gene column
gene_series = gwas_df['biomarker']
helper.populate_col(source = gene_series, target_col = 'gene')

# rs id column 
rs_id_series = gwas_df['SNPS'].apply(lambda x: x[2:])
helper.populate_col(source = rs_id_series, target_col = 'rs_id', dtype = 'Int64')

# evidence source column 
evidence_series = gwas_df['Evidence'] = 'GWAS Catalog|https://www.ebi.ac.uk/gwas/search?query=' + gwas_df['biomarker']
helper.populate_col(source = evidence_series, target_col = 'evidence_source')

# notes column 
notes_series = pd.Series('GRCh38', index = helper.biomarker_df.index)
helper.populate_col(source = notes_series, target_col = 'notes')

# disease name column 
disease_series = gwas_df['disease'].str.lower() 
helper.populate_col(source = disease_series, target_col = 'disease')

# assessed entity type column 
assessed_type_series = pd.Series('gene or protein', index = helper.biomarker_df.index)
helper.populate_col(source = assessed_type_series, target_col = 'assessed_entity_type')

# main_x_ref column 
helper.map_x_ref(prefix = 'dbSNP:rs')

# biomarker status column 
helper.set_biomarker_status(val = 'presence of')

# best biomarker type column
helper.set_best_biomarker_type(val = 'risk_biomarker')

# assessed entity type column
helper.set_assessed_entity_type(val = 'gene')

helper.biomarker_df

,biomarker_id,main_x_ref,assessed_biomarker_entity,biomarker_status,best_biomarker_type,specimen_type,loinc_code,condition_name,assessed_entity_type,evidence_source,notes,rs_id,gene,disease,uniprot,name,system,doid,mutation,variation
0,NaN,dbSNP:rs812481,rs812481 mutation in leucine-rich repeat prote...,presence of,risk_biomarker,nan,nan,colorectal cancer (DOID:9256.0),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,GRCh38,812481,LRIG1,colorectal cancer,Q5XWD3,Leucine-rich repeat protein LRIG1,nan,9256.0,NaN,NaN
1,NaN,dbSNP:rs73208120,rs73208120 mutation in alternative protein nos...,presence of,risk_biomarker,nan,nan,colorectal cancer (DOID:9256.0),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,GRCh38,73208120,NOS1,colorectal cancer,L8E6X1,Alternative protein NOS1,nan,9256.0,NaN,NaN
2,NaN,dbSNP:rs3184504,rs3184504 mutation in ataxin-2 (ATXN2),presence of,risk_biomarker,nan,nan,colorectal cancer (DOID:9256.0),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,GRCh38,3184504,ATXN2,colorectal cancer,V9GY86,Ataxin-2,nan,9256.0,NaN,NaN
3,NaN,dbSNP:rs3184504,rs3184504 mutation in sh2b adapter protein 3 (...,presence of,risk_biomarker,nan,nan,colorectal cancer (DOID:9256.0),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,GRCh38,3184504,SH2B3,colorectal cancer,R4GN84,SH2B adapter protein 3,nan,9256.0,NaN,NaN
4,NaN,dbSNP:rs4946260,"rs4946260 mutation in discoidin, cub and lccl ...",presence of,risk_biomarker,nan,nan,colorectal cancer (DOID:9256.0),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,GRCh38,4946260,DCBLD1,colorectal cancer,H0Y4G4,"Discoidin, CUB and LCCL domain-containing prot...",nan,9256.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7114,NaN,dbSNP:rs8034191,rs8034191 mutation in hydroxylysine kinase (HYKK),presence of,risk_biomarker,nan,nan,lung cancer (DOID:1324.0),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,GRCh38,8034191,HYKK,lung cancer,A0A0C4DGM4,Hydroxylysine kinase,nan,1324.0,NaN,NaN
7115,NaN,dbSNP:rs551610,rs551610 mutation in rims1 protein (RIMS1),presence of,risk_biomarker,nan,nan,lung cancer (DOID:1324.0),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,GRCh38,551610,RIMS1,lung cancer,Q3ZCW0,RIMS1 protein,nan,1324.0,NaN,NaN
7116,NaN,dbSNP:rs1051730,rs1051730 mutation in neuronal nicotinic acety...,presence of,risk_biomarker,nan,nan,lung cancer (DOID:1324.0),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,GRCh38,1051730,CHRNA3,lung cancer,Q6EWN2,"Neuronal nicotinic acetylcholine receptor, alp...",nan,1324.0,NaN,NaN
7117,NaN,dbSNP:rs8034191,rs8034191 mutation in hydroxylysine kinase (HYKK),presence of,risk_biomarker,nan,nan,lung cancer (DOID:1324.0),gene,GWAS Catalog|https://www.ebi.ac.uk/gwas/search...,GRCh38,8034191,HYKK,lung cancer,A0A0C4DGM4,Hydroxylysine kinase,nan,1324.0,NaN,NaN
